<a href="https://colab.research.google.com/github/boomyun713/114_homework/blob/main/Week6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

In [2]:
from datasets import load_dataset

# 使用已轉換好的版本
dataset = load_dataset(
    "FinanceInc/auditor_sentiment",  # 這是轉換後的版本
    #split="train"
)

print(dataset)

print("訓練集：", dataset['train'])
print("測試集：", dataset['test'])

# 查看資料集的欄位名稱與類型
print("訓練集欄位：", dataset["train"].column_names)
print("測試集欄位：", dataset["test"].column_names)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

dataset_infos.json:   0%|          | 0.00/800 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/327k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/80.9k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3877 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/969 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 3877
    })
    test: Dataset({
        features: ['sentence', 'label'],
        num_rows: 969
    })
})
訓練集： Dataset({
    features: ['sentence', 'label'],
    num_rows: 3877
})
測試集： Dataset({
    features: ['sentence', 'label'],
    num_rows: 969
})
訓練集欄位： ['sentence', 'label']
測試集欄位： ['sentence', 'label']


In [3]:
# 查看訓練集中前幾筆數據
print("\n訓練集前5筆：")
print(dataset["train"][:5])

# 查看測試集中前幾筆數據
print("\n測試集前5筆：")
print(dataset["test"][:5])

# 查看資料集中有哪些 Keys
print("\n資料集的 keys：", dataset.keys())


訓練集前5筆：
{'sentence': ["Altia 's operating profit jumped to EUR 47 million from EUR 6.6 million .", 'The agreement was signed with Biohit Healthcare Ltd , the UK-based subsidiary of Biohit Oyj , a Finnish public company which develops , manufactures and markets liquid handling products and diagnostic test systems .', 'Kesko pursues a strategy of healthy , focused growth concentrating on sales and services to consumer-customers .', 'Vaisala , headquartered in Helsinki in Finland , develops and manufactures electronic measurement systems for meteorology , environmental sciences , traffic and industry .', 'Also , a six-year historic analysis is provided for these markets .'], 'label': [2, 2, 2, 1, 1]}

測試集前5筆：
{'sentence': ["TeliaSonera TLSN said the offer is in line with its strategy to increase its ownership in core business holdings and would strengthen Eesti Telekom 's offering to its customers .", 'STORA ENSO , NORSKE SKOG , M-REAL , UPM-KYMMENE Credit Suisse First Boston ( CFSB ) ra

In [5]:
from collections import Counter # 計算列表中每個元素的出現次數

# label_map = {0: "Negative", 1: "Neutral", 2: "Positive"}
# 計算訓練集標籤出現次數
label_counts_train = Counter(dataset["train"]["label"])
print("\n訓練集標籤分佈：", label_counts_train)

# 計算測試集標籤出現次數
label_counts_test = Counter(dataset["test"]["label"])
print("測試集標籤分佈：", label_counts_test)


訓練集標籤分佈： Counter({1: 2320, 2: 1077, 0: 480})
測試集標籤分佈： Counter({1: 559, 2: 286, 0: 124})


In [7]:
import pandas as pd  # 結構化資料處理套件

# 轉換為 DataFrame
df_train = pd.DataFrame(dataset["train"])
df_test = pd.DataFrame(dataset["test"])

print("\n訓練集前5筆：")
print(df_train.head())

print("\n測試集前5筆：")
print(df_test.head())


訓練集前5筆：
                                            sentence  label
0  Altia 's operating profit jumped to EUR 47 mil...      2
1  The agreement was signed with Biohit Healthcar...      2
2  Kesko pursues a strategy of healthy , focused ...      2
3  Vaisala , headquartered in Helsinki in Finland...      1
4  Also , a six-year historic analysis is provide...      1

測試集前5筆：
                                            sentence  label
0  TeliaSonera TLSN said the offer is in line wit...      2
1  STORA ENSO , NORSKE SKOG , M-REAL , UPM-KYMMEN...      2
2  Clothing retail chain Sepp+ñl+ñ 's sales incre...      2
3  Lifetree was founded in 2000 , and its revenue...      2
4  Nordea Group 's operating profit increased in ...      2


In [8]:
# 檢查空值
print("\n訓練集空值：", df_train.isnull().sum())
print("測試集空值：", df_test.isnull().sum())

# 刪除重複值（如果有的話）
df_train.drop_duplicates(subset=["sentence"], inplace=True)
df_test.drop_duplicates(subset=["sentence"], inplace=True)

print("\n訓練集大小：", df_train.shape)
print("測試集大小：", df_test.shape)


訓練集空值： sentence    0
label       0
dtype: int64
測試集空值： sentence    0
label       0
dtype: int64

訓練集大小： (3872, 2)
測試集大小： (969, 2)


In [9]:
train_val_split = dataset["train"].train_test_split(test_size=0.2, seed=42)
train_dataset = train_val_split["train"]
val_dataset = train_val_split["test"]
test_dataset = dataset["test"]
print("\n資料集劃分：")
print(f"訓練集大小：{len(train_dataset)}")
print(f"驗證集大小：{len(val_dataset)} （從訓練集切出，用於訓練過程中的評估）")
print(f"測試集大小：{len(test_dataset)} （保留到最後才使用，避免資訊洩漏）")


資料集劃分：
訓練集大小：3101
驗證集大小：776 （從訓練集切出，用於訓練過程中的評估）
測試集大小：969 （保留到最後才使用，避免資訊洩漏）


In [10]:
from transformers import AutoTokenizer
model_name = "bert-base-chinese"  # 請修改這裡！

print(f"\n使用的模型：{model_name}")
from transformers import AutoTokenizer  # AutoTokenizer 可以根據指定的模型名稱，自動選擇和載入與該模型相匹配的分詞器

# 使用指定的 Tokenizer 進行 Tokenization(分詞)
tokenizer = AutoTokenizer.from_pretrained(model_name)
def tokenize_function(example):
    # 使用 tokenizer 將每個句子進行分詞
    # 設定 padding="max_length" 確保所有句子補齊至固定長度
    # 設定 truncation=True 以截斷超過最大長度的句子
    return tokenizer(example["sentence"], padding="max_length", truncation=True)

# 對訓練集、驗證集和測試集進行 Tokenize
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)


使用的模型：bert-base-chinese


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

Map:   0%|          | 0/3101 [00:00<?, ? examples/s]

Map:   0%|          | 0/776 [00:00<?, ? examples/s]

Map:   0%|          | 0/969 [00:00<?, ? examples/s]

In [11]:
print("\nTokenized 訓練集第一筆：")
print(tokenized_train[:1])

print("\nTokenized 驗證集第一筆：")
print(tokenized_val[:1])

print("\nTokenized 測試集第一筆：")
print(tokenized_test[:1])
# 確認 tokenized_datasets 是否包含需要的欄位
print("\n訓練集特徵：", tokenized_train.features)
print("驗證集特徵：", tokenized_val.features)
print("測試集特徵：", tokenized_test.features)


Tokenized 訓練集第一筆：
{'sentence': ["The financial impact is estimated to be some 1.5 MEUR annual improvement in the division 's result , starting from fiscal year 2007 ."], 'label': [2], 'input_ids': [[101, 100, 12200, 10481, 12624, 8310, 11696, 9007, 12125, 8168, 8228, 8815, 13048, 122, 119, 126, 100, 12276, 10665, 10481, 10538, 8519, 8631, 8217, 8174, 11308, 12556, 112, 161, 13170, 117, 10906, 8221, 8670, 8533, 10203, 8315, 9299, 8201, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [12]:
import torch  # 深度學習框架

# 如果有GPU就用GPU，沒有GPU用CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [13]:
from transformers import AutoModelForSequenceClassification  # 序列分類模型

# 載入模型（使用作業區域 1 選擇的模型）
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
model.to(device)  # 將模型移到對應裝置

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [14]:
from transformers import TrainingArguments  # 包裝訓練參數

training_args = TrainingArguments(
    output_dir="./results",          # 訓練結果的儲存位置，包括模型檔案和預測結果。
    eval_strategy="epoch",     # 設置評估策略為每個訓練輪次（epoch）結束後進行一次評估。
    learning_rate=2e-5,              # 設置學習率，這裡使用 2e-5，通常較小的學習率更適合微調預訓練模型。
    per_device_train_batch_size=4,   # 設定每個裝置（如每張 GPU）的訓練批次大小為 4。
    per_device_eval_batch_size=4,    # 設定每個裝置的評估批次大小為 4。
    num_train_epochs=3,              # 訓練輪次設定為 3，模型將完整遍歷訓練集三次。
    weight_decay=0.01,               # 設置權重衰減（L2正則化）參數，這裡為 0.01，用於防止過擬合。
    logging_dir="./logs",            # 日誌存放目錄，用於儲存 TensorBoard 或其他日誌。
    logging_steps=10,                # 設置每 10 個步驟記錄一次訓練信息，便於監控訓練過程。
    report_to="none",                 # 不將訓練日誌發送到任何平臺（如 TensorBoard），可以改為 "tensorboard" 以啟用。
)

In [15]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

def compute_metrics(pred):
    labels = pred.label_ids  # 取得真實值的id
    preds = np.argmax(pred.predictions, axis=1)  # 預測值找到最高機率的索引
    accuracy = accuracy_score(labels, preds)  # 計算準確率
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted")  # 計算其他指標
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [16]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,  # 使用訓練集
    eval_dataset=tokenized_val,     # 使用驗證集（從訓練集切出來的）
    compute_metrics=compute_metrics,
)

# 開始訓練
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.684400,0.841733,0.645619,0.591373,0.645619,0.586390
2,0.790300,0.776300,0.690722,0.661823,0.690722,0.631880
3,0.594400,0.776968,0.697165,0.628127,0.697165,0.659354


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


TrainOutput(global_step=2328, training_loss=0.7849442551226141, metrics={'train_runtime': 1253.9334, 'train_samples_per_second': 7.419, 'train_steps_per_second': 1.857, 'total_flos': 2447744125123584.0, 'train_loss': 0.7849442551226141, 'epoch': 3.0})

In [17]:
print("\n=== 驗證集評估結果 ===")
val_result = trainer.evaluate(eval_dataset=tokenized_val)
print(val_result)


=== 驗證集評估結果 ===


{'eval_loss': 0.7769684195518494, 'eval_accuracy': 0.6971649484536082, 'eval_precision': 0.6281269672701502, 'eval_recall': 0.6971649484536082, 'eval_f1': 0.6593540072540263, 'eval_runtime': 26.2748, 'eval_samples_per_second': 29.534, 'eval_steps_per_second': 7.384, 'epoch': 3.0}


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [18]:
# 評估模型在測試集的表現（最終評估）
print("\n=== 測試集評估結果（最終評估）===")
test_result = trainer.evaluate(eval_dataset=tokenized_test)
print(test_result)


=== 測試集評估結果（最終評估）===


{'eval_loss': 0.809520959854126, 'eval_accuracy': 0.6790505675954592, 'eval_precision': 0.7284953716585972, 'eval_recall': 0.6790505675954592, 'eval_f1': 0.6381406330555031, 'eval_runtime': 31.1334, 'eval_samples_per_second': 31.124, 'eval_steps_per_second': 7.805, 'epoch': 3.0}


In [19]:
print("\n=== 測試集句子展示 ===")
print("以下是測試集中的前 50 個句子：\n")

label_map = {0: "Negative", 1: "Neutral", 2: "Positive"}

for i in range(50):
    sentence = test_dataset[i]["sentence"]
    label = test_dataset[i]["label"]
    print(f"[{i}] {sentence}")
    print(f"    真實標籤：{label_map[label]}\n")


=== 測試集句子展示 ===
以下是測試集中的前 50 個句子：

[0] TeliaSonera TLSN said the offer is in line with its strategy to increase its ownership in core business holdings and would strengthen Eesti Telekom 's offering to its customers .
    真實標籤：Positive

[1] STORA ENSO , NORSKE SKOG , M-REAL , UPM-KYMMENE Credit Suisse First Boston ( CFSB ) raised the fair value for shares in four of the largest Nordic forestry groups .
    真實標籤：Positive

[2] Clothing retail chain Sepp+ñl+ñ 's sales increased by 8 % to EUR 155.2 mn , and operating profit rose to EUR 31.1 mn from EUR 17.1 mn in 2004 .
    真實標籤：Positive

[3] Lifetree was founded in 2000 , and its revenues have risen on an average by 40 % with margins in late 30s .
    真實標籤：Positive

[4] Nordea Group 's operating profit increased in 2010 by 18 percent year-on-year to 3.64 billion euros and total revenue by 3 percent to 9.33 billion euros .
    真實標籤：Positive

[5] Operating profit for the nine-month period increased from EUR3 .1 m and net sales increased fr

In [20]:
test_indices = [1, 7, 15, 9, 6]  # 請修改這裡！選擇你想測試的句子索引

print(f"\n你選擇測試的句子索引：{test_indices}")
# ============================================================================

# 根據選擇的索引，取得測試句子
test_texts = [test_dataset[i]["sentence"] for i in test_indices]
true_labels = [test_dataset[i]["label"] for i in test_indices]

print("\n=== 開始預測 ===\n")
# Tokenize 測試句子
test_encodings = tokenizer(test_texts, truncation=True, padding=True, return_tensors="pt").to(device)
outputs = model(**test_encodings)

# 取得預測結果
preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()  # 將結果轉回 CPU 以便處理

# 顯示預測結果
predicted_labels = [label_map[pred] for pred in preds]
true_labels_text = [label_map[label] for label in true_labels]

# 計算準確率
correct = sum([1 for true, pred in zip(true_labels_text, predicted_labels) if true == pred])
accuracy = correct / len(test_texts) * 100

print(f"你選擇的 {len(test_texts)} 個句子的預測結果：\n")

for i, (idx, text, true_label, pred_label) in enumerate(zip(test_indices, test_texts, true_labels_text, predicted_labels)):
    correct_mark = "✓ 正確" if true_label == pred_label else "✗ 錯誤"
    print(f"{i+1}. 測試集索引 [{idx}]")
    print(f"   句子：{text}")
    print(f"   真實標籤：{true_label}")
    print(f"   預測標籤：{pred_label}")
    print(f"   結果：{correct_mark}\n")

print(f"準確率：{correct}/{len(test_texts)} = {accuracy:.2f}%")


你選擇測試的句子索引：[1, 7, 15, 9, 6]

=== 開始預測 ===

你選擇的 5 個句子的預測結果：

1. 測試集索引 [1]
   句子：STORA ENSO , NORSKE SKOG , M-REAL , UPM-KYMMENE Credit Suisse First Boston ( CFSB ) raised the fair value for shares in four of the largest Nordic forestry groups .
   真實標籤：Positive
   預測標籤：Positive
   結果：✓ 正確

2. 測試集索引 [7]
   句子：In January-September 2010 , Fiskars ' net profit went up by 14 % year-on-year to EUR 65.4 million and net sales to EUR 525.3 million from EUR 487.7 million .
   真實標籤：Positive
   預測標籤：Positive
   結果：✓ 正確

3. 測試集索引 [15]
   句子：Adjusted for changes in the Group structure , the Division 's net sales increased by 1.7 % .
   真實標籤：Positive
   預測標籤：Positive
   結果：✓ 正確

4. 測試集索引 [9]
   句子：Nyrstar has also agreed to supply to Talvivaara up to 150,000 tonnes of sulphuric acid per annum for use in Talvivaara 's leaching process during the period of supply of the zinc in concentrate .
   真實標籤：Positive
   預測標籤：Positive
   結果：✓ 正確

5. 測試集索引 [6]
   句子：The Lithuanian beer market made up 14.41 milli